Note: this notebook will be mainly used for the capstone project

In [5]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Scrap data from Wikipedia page into a DataFrame

In [10]:
html = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
bs = BeautifulSoup(html, 'html.parser')

tabs = []
# find the td label
for tr in bs.table.find_all('tr')[1:]:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        tabs.append(row)
# convert to data_frame
df = pd.DataFrame(tabs, columns=["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Process the data_frame: 
1. Drop the nan rows in Borough column
2. Group neighborhoods in the same borough
3. For Neighborhood Not assigned, replace the value with Borough

In [11]:
df = df[df.Borough !='Not assigned']  # drop the nan rows in Borough column
df_combine = df.groupby('PostalCode').agg({'Borough': lambda x: x.unique(), 'Neighborhood':lambda x: ','.join(x)}).reset_index() # combine rows which have the same PostalCode
df_combine.loc[df_combine.Neighborhood == 'Not assigned', 'Neighborhood'] = df_combine.loc[df_combine.Neighborhood == 'Not assigned', 'Borough'] # assign missing values for Neighborhood
df_combine.shape

(103, 3)

## test the result with assignment

In [14]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

test_df = df_combine[df_combine['PostalCode'].isin(test_list)]
    
test_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
5,M1J,Scarborough,Scarborough Village
11,M1R,Scarborough,"Maryvale,Wexford"
17,M2H,North York,Hillcrest Village
35,M4B,East York,"Woodbine Gardens,Parkview Hill"
38,M4G,East York,Leaside
43,M4M,East Toronto,Studio District
53,M5A,Downtown Toronto,Harbourfront
57,M5G,Downtown Toronto,Central Bay Street
68,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf..."
